
![Py4Eng](img/logo.png)

# Object Oriented Programming

# Introduction to OOP


*Everything in Python is an object* so everything in Python has a class. 

You can find out the type of an object by calling the function `type`.

Even the return of the `type` function has a type: it's `type`!

In [1]:
type(5)

int

In [2]:
a = 'Hi'
type_a = type(a)
print(type_a)
type_type_a = type(type(a))
print(type_type_a)

<class 'str'>
<class 'type'>


The basic object type is `object`:

In [3]:
print(object)
print(type(object))

<class 'object'>
<class 'type'>


Every class implicitly inherits from `object`, so all classes have some default attributes:

In [4]:
object_dir = dir(object)
for att in object_dir:
    print(att)

__class__
__delattr__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__


Other types introduce new attributes:

In [5]:
int_dir = dir(int)

for att in int_dir:
    if att not in object_dir:
        print(att)

__abs__
__add__
__and__
__bool__
__ceil__
__divmod__
__float__
__floor__
__floordiv__
__getnewargs__
__index__
__int__
__invert__
__lshift__
__mod__
__mul__
__neg__
__or__
__pos__
__pow__
__radd__
__rand__
__rdivmod__
__rfloordiv__
__rlshift__
__rmod__
__rmul__
__ror__
__round__
__rpow__
__rrshift__
__rshift__
__rsub__
__rtruediv__
__rxor__
__sub__
__truediv__
__trunc__
__xor__
as_integer_ratio
bit_length
conjugate
denominator
from_bytes
imag
numerator
real
to_bytes


Those methods that `int` has but `object` doesn't have define the behaviour of `int`.
Specificially, all the `__xxx__` methods are special methods that interact with Python in specific ways, defining operators etc.

# _Point_ type

Let's learn more by writing a new type -- a class -- for a point in a 2-dimensional Euclidian space ($\mathbb{R}^2$).

We start with the class definition (`class`) and the constructor (`__init__`) which initialized the attributes of the class instance. 

We'll start with a 2D point, but we'll change it later to nD point.

Note that the first argument to methods (member functions) is always `self`, a reference to the instance.

We don't need to check the types of the `__init__` arguments; rather, we try to convert them to `float`, and let the conversion fail if the types are wrong.

In [7]:
class Point:
    """A point in 2D Euclidian plane.
    """
    
    def __init__(self, x, y):        
        # assert isinstance(x, (int, float)) and isinstance(y, (int, float))
        self.x = float(x)
        self.y = float(y)

In [8]:
origin = Point(0, 0)
print("origin", origin.x, origin.y)

p = Point(1, 2)
print("point", p.x, p.y)

origin 0.0 0.0
point 1.0 2.0


Notice that when we send a `Point` to the console we get:

In [9]:
p

Which is not useful, so we will define how `Point` is represented in the console by implementing `__repr__`  and `__str__`.

In [10]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)

In [11]:
p = Point(1,2)
p

Point(1.0, 2.0)

In [12]:
print(p)

(1.0, 2.0)


It is also very useful to be able to iterate over the coordinates. 
We implement this by returning the iterator of a tuple.

In [13]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)

In [14]:
p = Point(1,2)
for coor in p:
    print(coor)

1.0
2.0


Next we define a method to add two points. Addition is by elements, for a 2D point: $(x_1, y_1) + (x_2, y_2) = (x_1+x_2, y_1+y_2)$.

We also allow to add an `int` or `float`, in which case we add the point to a another point with both coordinates equal to the argument value.

In [15]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def add(self, other):
        self_type = type(self) # self_type is Point
        
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        
        else:
            raise TypeError(
            "unsupported operand type(s) for +: '{}' and '{}'".format(
                self_type.__name__, type(other).__name__))

In [16]:
Point(1,1).add(Point(2,2))

Point(3.0, 3.0)

In [17]:
Point(1, 1).add(2)

Point(3.0, 3.0)

In [18]:
Point(1,1).add('2')

TypeError: unsupported operand type(s) for +: 'Point' and 'str'

A nicer way to implement addition is to **overload** the addition operator `+` by implementing `__add__`, which is a name Python reserves for addition operations (those are double underscores). In that case, we don't need to raise the `TypeError` exception but rather just return the `NotImplemented` constant and Python will take care of the error (after checking if the right hand object knows how to do right-add using `__radd__`):

In [19]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
            
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented

In [20]:
Point(1,1) + Point(2,2)

Point(3.0, 3.0)

In [21]:
Point(1, 1) + 2

Point(3.0, 3.0)

In [22]:
Point(1, 1) + '2'

TypeError: unsupported operand type(s) for +: 'Point' and 'str'

We want to be a able to compare `Point`s:

In [23]:
Point(1,2) == Point(2,1)

False

In [24]:
Point(1,2) == Point(1,2)

False

In [25]:
p = Point(0, 0)
p == p

True

In [26]:
Point(1,2) > Point(2,1)

TypeError: '>' not supported between instances of 'Point' and 'Point'

So the default `==` checks by identity and `>` is not defined. 

We should overload both these operators by implementing `__eq__` and `__gt__`.

We do this by converting the point to a tuple and __delegating__ `__eq__` to tuple.
The conversion is handled by `__iter__`.

We can't delegate `__gt__` so we must implement it.

In [27]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
            
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y

First we check if two points are equal:

In [28]:
Point(1,0) == Point(1,2)

False

In [29]:
Point(0,1) == (0,1)

True

Then if one is *strictly* smaller than the other:

In [30]:
Point(1,0) > Point(1,2)

False

In [31]:
Point(5,6) > Point(1,2)

True

Note that by implementing `==` and `>` we have logically defined all other comparison operators. 
Indeed, Python fills in the other comparison operators:

In [32]:
Point(5,6) < Point(1,4)

False

The addition operator `+` returns a **new instance**.

In [33]:
p = Point(1, 1)
pp = p
p += Point(2, 0)
print(p)
print(pp)

(3.0, 1.0)
(1.0, 1.0)


Next we will write a method that implements `+=` efficiently, so that instead of returning a new instance, it changes the current instance **in-place**. 

This is called an [augmented arithmetic assignments](https://docs.python.org/3.5/reference/datamodel.html?highlight=__iadd__#object.__iadd__). These methods should return their result.

In [34]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self # always return self from __i???__
        elif isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        else:
            return NotImplemented

In [35]:
p = Point(1, 1)
pp = p
p + Point(1,2)
print(p)
p += Point(1,2)
print(p)
print(pp)

(1.0, 1.0)
(2.0, 3.0)
(2.0, 3.0)


We now write a method that given many points, checks if the current point is more extreme than the other points.

**Note:** specifying a function argument with a `*` before its name says that we can give zero or more values and they will be packed in a `tuple`.

In [36]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self # always return self from __i???__
        elif isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)

In [37]:
p = Point(5, 6)
p.is_extreme(Point(1,1))

True

In [38]:
p.is_extreme(Point(1,1), Point(2,5), Point(6,2))

False

We can also use the method via the class instead of the instance, and give the instance of interest (the one that we want to know if it is the extreme) as the first argument `self`. Similarly, we can either do `'hi'.upper()` or `str.upper('hi')`.

In [39]:
Point.is_extreme(Point(7,8), Point(1,1), Point(4,5), Point(2,3))

True

Let's add a method to return a polar representation of `Point`. We'll need to calculate the magnitude, which we might as well do with the `__abs__` method (corresponding to `abs`) and an `angle` method:

In [41]:
import math

class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self # always return self from __i???__
        elif isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        # optimized for 2D
        return math.hypot(self.x, self.y)
    
    def angle(self):
        return math.atan2(self.y, self.x)

    def to_polar(self):
        return abs(self), self.angle()        

In [42]:
print(Point(1, 0).to_polar())
print(Point(-1, 0).to_polar())
print(Point(1, 1).to_polar())
print(Point(2, 2).to_polar())

(1.0, 0.0)
(1.0, 3.141592653589793)
(1.4142135623730951, 0.7853981633974483)
(2.8284271247461903, 0.7853981633974483)


If we have `to_polar` that returns a tuple of polar coordinates, we might want to have `frompolar` that creates a new instance from a tuple of polar coordinates. 

This is called an **alternative constructor** and must be implemented as a **class method** as we don't have an instance at the time we call the method:

In [43]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self # always return self from __i???__
        elif isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        # optimized for 2D
        return math.hypot(*self) # "*self" is tuple unpacking
    
    def angle(self):
        return math.atan2(self.y, self.x)

    def to_polar(self):
        return abs(self), self.angle()    
    
    @classmethod
    def from_polar(cls, radius, angle): # note that the first argument is a class rather than instance
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)

In [44]:
p1 = Point(3, 4.5)
r, a = p1.to_polar()
p2 = Point.from_polar(r, a)
p1, p2, p1 == p2

(Point(3.0, 4.5), Point(3.0000000000000004, 4.5), False)

We got a `False` because there is some numerical error.

In [45]:
p1 = Point(1, 1)
r, a = p1.to_polar()
p2 = Point.from_polar(r * 2, a)
p1, p2

(Point(1.0, 1.0), Point(2.0000000000000004, 2.0))

We can also implement `__complex__` to allow to convert our `Point` to a complex number:

In [46]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)
        
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
    
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, (int, float)):
            self.x += other
            self.y += other
            return self # always return self from __i???__
        elif isinstance(other, type(self)):
            self.x += other.x
            self.y += other.y
            return self
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        # optimized for 2D
        return math.hypot(*self) # "*self" is tuple unpacking
    
    def angle(self):
        return math.atan2(self.y, self.x)

    def to_polar(self):
        return abs(self), self.angle()               
    
    @classmethod
    def from_polar(cls, radius, angle): # note that the first argument is a class rather than an instance
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
    
    def __complex__(self):
        return complex(*self)

In [47]:
complex(Point(1,1))

(1+1j)

## Immutable

We might want to make our `Point` immutable.
This is easy enough, we just need to change our coordinates `x` and `y` to `_x` and `_y` to signal (by convention) that it should not be used directly, and add `x` and `y` properties.

Note: We shouldn't implement `__iadd__`, though, in this case.

In [48]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, x, y):        
        self._x = float(x)
        self._y = float(y)
        
    @property
    def x(self):
        return self._x
    
    @property
    def y(self): 
        return self._y
        
    def __repr__(self):
        # what a programmer wants to see, should work with eval
        return "{}({}, {})".format(type(self).__name__, self.x, self.y)
        
    def __str__(self):
        # what a user wants to see
        return "({}, {})".format(self.x, self.y)
    
    def __iter__(self):
        tup = (self.x, self.y)
        return iter(tup)

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return self.x > other.x and self.y > other.y
        
    def __add__(self, other):
        self_type = type(self)
        if isinstance(other, (int, float)):
            return self_type(self.x + other, self.y + other)
        elif isinstance(other, self_type):
            return self_type(self.x + other.x, self.y + other.y)
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        # optimized for 2D
        return math.hypot(*self) 
    
    def angle(self):
        return math.atan2(self.y, self.x)

    def to_polar(self):
        return abs(self), self.angle()        
    
    @classmethod
    def from_polar(cls, radius, angle): # note that the first argument is a class rather than an instance
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
    
    def __complex__(self):
        return complex(*self)

In [49]:
p = Point(10, 100)
print(p)
p.x

(10.0, 100.0)


10.0

Note that we didn't have to change any other method (except remove `__iadd__`) because the `@property` decorator turns the `x` and `y` methods to instance members.

However, they cannot be directly changed now, making our class immutable, at least by convention:

In [50]:
p.x = 2

AttributeError: can't set attribute

## Bonus: N-Dimensional point

At this point, we can generalize our class to a point in n-dimensional space.

This requires only minor changes (except for some functions that don't work for more than two dimensions, like `__complex__`).

In [51]:
class Point:
    """A point (x,y) in the Euclidian plane.
    """
    
    def __init__(self, *coords): # instead of saving two floats for the coordinates, we save a tuple of floats
        self._coords = tuple(float(x) for x in coords)
    
    @property
    def coords(self):
        return self._coords
    
    @property
    def x(self): # access coords
        return self.coords[0]
    
    @property
    def y(self): # access coords
        return self.coords[1]
    
    def __len__(self): # delegate to tuple
        return len(self.coords)
    
    def __repr__(self): # delegate to str
        # what a programmer wants to see, should work with eval
        return "{}{}".format(type(self).__name__, str(self))
        
    def __str__(self): # implement with str.join
        # what a user wants to see
        coord_str = (str(x) for x in self)
        coord_str = str.join(', ', coord_str)
        return "({})".format(coord_str)
    
    def __iter__(self): # delegate to tuple
        return iter(self.coords) 

    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(x1 > x2 for x1, x2 in zip(self, other)) # zip over coords
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, (int, float)):
            coords = (x + other for x in self) # iterate over coords
            return cls(*coords) # *coords unpacks coords
        elif isinstance(other, cls):
            if len(self) != len(other): 
                raise NotImplementedError("Wrong number of dimensions: {} and {}".format(len(self), len(other)))
            coords = (x1 + x2 for x1, x2 in zip(self, other)) # zip over coords
            return cls(*coords) # *coords unpacks coords
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, numbers.Real):
            self.coords = (x + other for x in self)
            return self 
        elif isinstance(other, type(self)):
            if len(self) != len(other): raise NotImplementedError("Wrong number of dimensions: {} and {}".format(len(self), len(other)))
            self.coords = (x1 + x2 for x1, x2 in zip(self, other))
            return self
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        if len(self) == 2:
            return math.hypot(*self) # optimized for 2D
        else:
            return math.sqrt(sum(x*x for x in self)) # general purpose
    
    def angle(self):
        if len(self) == 2:
            return math.atan2(self.y, self.x)
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()  
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))
    
    @classmethod
    def from_polar(cls, radius, angle): # note that the first argument is a class rather than an instance
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
    
    def __complex__(self):
        if len(self) == 2:
            return complex(*self)
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))

In [52]:
p = Point(1, 2, 3, 4)
print(p)

(1.0, 2.0, 3.0, 4.0)


In [53]:
p + Point(5, 5, 0, 0)

Point(6.0, 7.0, 3.0, 4.0)

In [54]:
abs(p)

5.477225575051661

We can also define the `__getitem__` method so that we can get specific coordinates (we'll not define the `__setitem__` to keep the class "imutable", but of course this can be done).

By having `__len__` and `__getitem__` we are implementing the *sequence protocol*, and therefore we get `__iter__` and `__reversed__` for free, so we can throw `__iter__` away (although we might want to keep it if we think our implementation is better).

In [55]:
class Point:
    """A point in a Euclidian space.
    """
    
    def __init__(self, *coords):        
        self._coords = tuple(float(x) for x in coords)
    
    @property
    def coords(self):
        return self._coords

    def __getitem__(self, key):
        return self.coords[key]
        
    def __len__(self): # delegate to tuple
        return len(self.coords)
    
    ## __iter__ not needed

    @property
    def x(self): # access coords
        return self[0]
    
    @property
    def y(self): # access coords
        return self[1]
        
    def __repr__(self): 
        # what a programmer wants to see, should work with eval
        return "{}{}".format(type(self).__name__, str(self))
        
    def __str__(self):
        # what a user wants to see
        coord_str = (str(x) for x in self)
        coord_str = str.join(', ', coord_str)
        return "({})".format(coord_str)
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __gt__(self, other):
        return all(x1 > x2 for x1, x2 in zip(self, other))
    
    def __add__(self, other):
        cls = type(self)
        if isinstance(other, numbers.Real):
            coords = (x + other for x in self) 
            return cls(*coords) 
        elif isinstance(other, cls):
            if len(self) != len(other): 
                raise NotImplementedError("Wrong number of dimensions: {} and {}".format(len(self), len(other)))
            coords = (x1 + x2 for x1, x2 in zip(self, other))
            return cls(*coords) 
        else:
            return NotImplemented
    
    def is_extreme(self, *points):
        return all(self > point for point in points)
    
    def __abs__(self):
        if len(self) == 2:
            return math.hypot(*self) # optimized for 2D
        else:
            return math.sqrt(sum(x*x for x in self)) # general purpose
    
    def angle(self):
        if len(self) == 2:
            return math.atan2(self.y, self.x)
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))

    def to_polar(self):
        if len(self) == 2:
            return abs(self), self.angle()  
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))
    
    @classmethod
    def from_polar(cls, radius, angle): 
        x = radius * math.cos(angle)
        y = radius * math.sin(angle)
        return cls(x, y)
    
    def __complex__(self):
        if len(self) == 2:
            return complex(*self)
        else:
            raise NotImplementedError("Not implemented for {} dimensions".format(len(self)))

In [56]:
p = Point(1, 2, 3, 4, 5)
p.coords

(1.0, 2.0, 3.0, 4.0, 5.0)

In [57]:
p[0], p[1]

(1.0, 2.0)

In [58]:
p[-1]

5.0

In [59]:
p[10]

IndexError: tuple index out of range

In [60]:
p.x, p.y

(1.0, 2.0)

## Bonus: __dict__

An object attributes are saved in a dictionary called `__dict__`:

In [61]:
p.__dict__

{'_coords': (1.0, 2.0, 3.0, 4.0, 5.0)}

However, a dictionary has some overhead, and a more memory efficient way to save the attributes is done by defining the class attribute `__slots__`.

Let's see an example for 2D point:

In [62]:
class RegularPoint:
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({}, {})".format(type(self).__name__, *self)
    
class EfficientPoint:
    __slots__ = ('x', 'y')
    
    def __init__(self, x, y):        
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        yield self.x
        yield self.y
    
    def __repr__(self):
        return "{}({}, {})".format(type(self).__name__, *self)

We load [ipython_memory_usage](https://github.com/ianozsvald/ipython_memory_usage), a package that monitors memory usage inside the notebook.

Install from with `pip install ipython_memory_usage`, and on Windows you will also n
`conda install psutil`.

In [63]:
import ipython_memory_usage.ipython_memory_usage as imu
imu.start_watching_memory()

In [63] used 0.0078 MiB RAM in 0.17s, peaked 0.00 MiB above current, total RAM usage 57.71 MiB


In [64]:
lst1 = [RegularPoint(0, 0) for _ in range(10**7)]

In [64] used 2179.7891 MiB RAM in 9.27s, peaked 14.20 MiB above current, total RAM usage 2237.50 MiB


In [65]:
lst2 = [EfficientPoint(0, 0) for _ in range(10**7)]

In [65] used 1236.9336 MiB RAM in 7.69s, peaked 0.00 MiB above current, total RAM usage 3474.43 MiB


So we see that for $10^7$ points, the `__slots__` class requires ~2-fold less memory.

We should now delete the references to release these GB of RAM with the `del` statement:

In [66]:
del lst1
del lst2

In [66] used -3328.0508 MiB RAM in 1.29s, peaked 3328.07 MiB above current, total RAM usage 146.38 MiB


And finally stop monitoring the memory:

In [67]:
imu.stop_watching_memory()

# __Rectangle class__

We will implement two classes for rectangles, and compare the two implementations.

## First implementation - two points

The first implementation defines a rectangle by its lower left and upper right vertices.

In [68]:
class Rectangle1:
    """Describe a parallel-axes rectangle by storing two points.
    
    Attributes
    ----------
    llv : Point
        lower left vertex
    urv : Point
        upper right vertex
    """
    
    def __init__(self, lower_left_vertex, upper_right_vertex):        
        self.llv = Point(*lower_left_vertex)
        self.urv = Point(*upper_right_vertex)
        assert self.llv < self.urv, "Lower left vertex should be lower than upper right vertex"
        
    def __str__(self):
        return "Rectangle with lower left {} and upper right {}".format(self.llv, self.urv)

    @property
    def height(self):
        return self.urv.y - self.llv.y
      
    @property
    def width(self):
        return self.urv.x - self.llv.x

    @property
    def area(self):
        return self.height * self.width
       
    def transpose(self):
        """Reflection with regard to the line passing through lower left vertex with angle 315 (-45) degrees
        """
        x, y = self.llv
        llv = Point(x - self.height, y - self.width)
        cls = type(self)
        return cls(llv, self.urv)

In [69]:
rect = Rectangle1(Point(0, 0), Point(2,1))
print(rect)
print("Area:", rect.area)
print("Dimensions:", rect.height, rect.width)
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left (0.0, 0.0) and upper right (2.0, 1.0)
Area: 2.0
Dimensions: 1.0 2.0
Transposed: Rectangle with lower left (-1.0, -2.0) and upper right (2.0, 1.0)


## Exercise: contains

Overload the `__contains__` method of the rectangle class so that given a point it will return `True` if the point is inside the rectangle and `False` otherwise.

In [71]:
class Rectangle1:
   # your code here

In [72]:
rect = Rectangle1(Point(0, 0), Point(2, 2))
assert Point(1, 1) in rect
assert Point(1.5, 0.5) in rect
assert Point(3, 1) not in rect
assert Point(0.5, 2.5) not in rect

## Exercise: Rectangle2

The second implementation defines a rectangle by the lower left point, the height and the width.

Define the exact same methods as in `Rectangle1`, with the same input and output, but  different inner representation / implementation.

In [73]:
class Rectangle2:
    """Describe a parallel-axes rectangle by storing lower left point, height and width.
    
    Attributes
    ----------
    point : Point
        lower left point
    width : float
        width
    height : float
        height
    """    
   
    # your code here

In [74]:
rect = Rectangle2(Point(0, 0), 1, 2)
print(rect)
print("Area:", rect.area)
print("Dimensions:", rect.width, rect.height)
t_rect = rect.transpose()
print("Transposed:", t_rect)

Rectangle with lower left (0.0, 0.0), width 1, and height 2
Area: 2
Dimensions: 1 2
Transposed: Rectangle with lower left (0.0, 0.0), width 2, and height 1


# Inheritance

Next we will see how inheritance works in Python.

We define a `Door` class. The door is either open or closed, it can be opened or closed, and it can be represented as a string.

This example also shows how to define a setter property. 

In [8]:
class Door:
    OPEN = 'open'
    CLOSED = 'closed'
    
    def __init__(self, status=CLOSED):
        self._status = status
            
    def __repr__(self):
        return '{}({})'.format(type(self).__name__, self._status)

    def __str__(self):
        return 'Door {0} is {1}'.format(id(self), self._status)
    
    def open(self):
        self._status = self.OPEN
        
    def close(self):
        self._status = self.CLOSED

In [9]:
door = Door()
print(door)
door

Door 4551840384 is closed


Door(closed)

Now we want to define a secure door which only opens with a password.

The secure door inherits from the regular door, but it makes some changes:

- `__init__` accepts a password and saves it's hash as an attribute
- `open` requires the password from the user to actually open the door

The `super` function gives access to the parent class.

In [10]:
from getpass import getpass
from hashlib import sha224

def digest(password):
    """Hash a password using sha224 algorithm.
    """
    return sha224(password.encode('utf8')).hexdigest()

In [11]:
class SecurityDoor(Door):
    """A door that requires a password to open.
    """
  
    def __init__(self, password):
        super().__init__()
        self.secret = digest(password)
        
    def open(self):
        if digest(getpass("What is the password?")) == self.secret:
            super().open()
        else:
            print("Wrong password!")

In [12]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password? ··········


Door 4551843216 is open
Door 4551843216 is closed


Multiple inheritance works in Python. If there are clashes between names Python will search from the first parent to the last. 

However, multiple inheritance and inheritance in general are many times not neccessary and similar results can be produced by composition and delegation, without creating the complexity and constraints that inheritance induces.

In the words of the famous [Design Patterns](https://en.wikipedia.org/wiki/Composition_over_inheritance) book:

> Favor 'object composition' over 'class inheritance'.

# Composition and delegation

Composition means that one object **explicitly delegates** some tasks to another object. 

Delegation is implicit in inheritance, and explicit in composition.

The *Zen of Python* (`import this`) says that *Explicit is better than implicit*, so we should conclude that composition is better than inheritance.

Let's implement regular composition, which simply makes an object part of the other as an attribute:

In [15]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self._door = Door(status=Door.CLOSED)
        self._secret = digest(password)
    
    def __repr__(self):
        return repr(self._door)
               
    def open(self):
        if digest(getpass("What is the password?")) == self._secret:
            self._door.open()
        else:
            print("Wrong password!")
        
    def close(self):
        self._door.close()

In [17]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password? ··········


Door(open)
Door(closed)




The primary goal of composition is to relax the coupling between objects. 
This little example shows that now `SecurityDoor` is an `object` and no more a `Door`, 
which means that the internal structure of `Door` is not copied. 
For this very simple example both `Door` and `SecurityDoor` are not big classes, 
but in a real system objects can be very complex; 
this means that their allocation consumes a lot of memory and if a system contains thousands or millions of objects that could be an issue.

The composed `SecurityDoor` has to redefine every attribute since the concept of delegation applies only to methods and not to attributes, doesn't it?

**No.** Python allows objects manipulation and attribute access is one of the most useful. 
Accessing attributes is ruled by a special method called `__getattribute__` that is called whenever an attribute of the object is accessed. 
Overriding `__getattribute__`, however, is overkill; 
it is a very complex method, and, being called on every attribute access, any change makes the whole thing slower.

The method we have to leverage to delegate attribute access is `__getattr__`, which is a special method that is called whenever the requested attribute is not found in the object. 
So basically it is the right place to dispatch all attribute and method access our object cannot handle. 

The previous example becomes:

In [20]:
class SecurityDoor:
    """A door that requires a password to open.
    """
    
    def __init__(self, password):
        self._door = Door(status=Door.CLOSED)
        self._secret = digest(password)
                          
    def __repr__(self):
        return repr(self._door)
    
    def open(self):
        if digest(getpass("What is the password?")) == self._secret:
            self._door.open()
        else:
            print("Wrong password!")
        
    def __getattr__(self, attr):
        return getattr(self._door, attr)

In [21]:
secure_door = SecurityDoor('opensesame')
secure_door.open()
print(secure_door)
secure_door.close()
print(secure_door)

What is the password? ··········


Door(open)
Door(closed)


As this last example shows, delegating every member access through `__getattr__` is very simple. 
Pay attention to `getattr` which is different from `__getattr__`. 
The former is a built-in function that is equivalent to the dotted syntax, i.e. `getattr(obj, 'someattr')` is the same as `obj.someattr`, but you have to use it since the name of the attribute is contained in a string.

**Composition provides a superior way to manage delegation** since it can selectively delegate the access, even mask some attributes or methods, while inheritance cannot. 

In Python you also avoid the memory problems that might arise with delegation when you put many objects inside another; Python handles everything through its reference, so the size of an attribute is constant and very limited.

## Exercise: revolving door

Define a new class, `RevolvingDoor`, which closes immediately after it is opened. Define it once with inheritance and once with composition.

In [86]:
class RevolvingDoor(Door):
    # you code here
    
rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

Door 4530430256 is closed


In [88]:
class RevolvingDoor:
    # your code here
    
rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

Door(status=closed)


# Bonus: Polymorphism

In Python, polymorphism is baked into the language, due to the **Duck typing** principle. 

We saw above how this relates to methods like `__add__`, `__sub__`, `__repr__`, and `__contains__`.

Another example is a file object:

In [89]:
f = open("../data/crops.txt")
print(type(f))
print(dir(f))
f.close()

<class '_io.TextIOWrapper'>
['_CHUNK_SIZE', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_finalizing', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'reconfigure', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'write_through', 'writelines']


Say we write a function that gets a file and returns all the lines that start with a given prefix (say, 'Am'):

In [90]:
def filter_by_prefix(file, prefix):
    return [line.strip() for line in file if line.startswith(prefix)]

with open('../data/crops.txt', 'r') as f:
    print(filter_by_prefix(f, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


But now, say we want to read from `crops.txt.gz`, which is compressed with gzip. There is a module for reading gzipped files:

In [91]:
import gzip

In [92]:
gzfile = gzip.open('../data/crops.txt.gz' ,'r')
print(type(gzfile))
print(dir(gzfile))
print(isinstance(gzfile, f.__class__))
gzfile.close()

<class 'gzip.GzipFile'>
['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_impl', '_buffer', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_can_read', '_check_can_seek', '_check_can_write', '_check_not_closed', '_init_write', '_write_gzip_header', 'close', 'closed', 'detach', 'filename', 'fileno', 'fileobj', 'flush', 'isatty', 'mode', 'mtime', 'myfileobj', 'name', 'peek', 'read', 'read1', 'readable', 'readinto', 'readinto1', 'readline', 'readlines', 'rewind', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'writelines']
False


You notice that this is a different type than a file we opened with `open` (that would be `f`), and this is not even inheritance relationship (we know this because `isinstace` returned `False`). 

But our function `filter_by_prefix` doesn't care about the type, all it wants is an **object that you can loop over with a `for`**: an iterable object that implements the `__iter__` method by either:

- returning an iterator, which is an object that has a `next` method and stops iteration by raising a `StopIteration` exception, or 
- using the `yield` statement, which creates a generator.

Indeed, `gzfile` implements `__iter__`:

In [93]:
hasattr(gzfile, '__iter__')

True

So we can use it with our function:

In [94]:
with gzip.open('../data/crops.txt.gz', 'rt') as gzfile: # rt is for reading text
    print(filter_by_prefix(gzfile, 'Am'))

['Amaranthus cruentus', 'Amaranthus spp.', 'Amaranthus viridis', 'Amomum subulatum', 'Amorphophallus konjac', 'Amorphophallus paeoniifolius']


It doesn't even have to be a file. A list is just as good:

In [95]:
continents = ['America', 'Europe', 'Asia', 'Africa', 'Anarctica', 'Australia']
print(filter_by_prefix(continents, 'Am'))

['America']


# Bonus: Destructor

Python uses a garbase collector, and the standard CPython implementation uses reference counting and deletes objects when there are no more references to them.

You can remove references using the `del` command (see in [Python Tutor](http://pythontutor.com/visualize.html#code=a+%3D+%5B1,2,3%5D%0Adel+a&mode=display&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=3&rawInputLstJSON=%5B%5D&curInstr=2)):

In [96]:
a = [1,2,3]
del a
a

NameError: name 'a' is not defined

You can implement the [`__del__`](https://docs.python.org/3.5/reference/datamodel.html#object.__del__) method which is called when an instance is about to be destroyed. 
This is also called a **destructor**. 
Similar to `__init__`, if a base (parent) class has a `__del__` method then you must explicitly call it. 
It is not guaranteed that __del__() methods are called for objects that still exist when the interpreter exits.

In [97]:
class A:
    def __init__(self, name):
        self.name = name
        print('Initialize')
        
    def __del__(self):
        print('Delete')
        
    def __repr__(self):
        return self.name

In [98]:
a = A('Alex')
print(a)

Initialize
Alex


In [99]:
b = a
print(b)

Alex


In [100]:
del a
print(b)

Alex


In [101]:
del b

Delete


Note that several things can implicitly save a reference to your object, including the `_` variable name and stack traces.

# References

- Python 3 [OOP tutorial](https://docs.python.org/3/tutorial/classes.html)
- Leonardo Giordani's [OOP notebooks](http://nbviewer.jupyter.org/github/lgiordani/blog_source/blob/master/pelican/content/notebooks/Python_3_OOP_Part_3__Delegation__composition_and_inheritance.ipynb)
- [ Fluent Python](http://shop.oreilly.com/product/0636920032519.do) by Luciano Ramalho is a great book for intermediate and advanced Python programmers.
- [Python data model docs](https://docs.python.org/3.5/reference/datamodel.html).

# Solutions

## Solution: contains

In [ ]:
class Rectangle1:
   
    def __init__(self, lower_left_vertex, upper_right_vertex):        
        self.llv = Point(*lower_left_vertex)
        self.urv = Point(*upper_right_vertex)
        assert self.llv < self.urv, "Lower left vertex should be lower than upper right vertex"
        
    def __contains__(self, point):
        return self.llv < point < self.urv

## Solution: Rectangle2

In [ ]:
class Rectangle2:
    """Describe a parallel-axes rectangle by storing lower left point, height and width.
    
    Attributes
    ----------
    point : Point
        lower left point
    width : float
        width
    height : float
        height
    """    
    def __init__(self, lower_left_vertex, width, height):        
        self.llv = Point(*lower_left_vertex)
        self._height = height
        self._width = width
                
    def __str__(self):
        return "Rectangle with lower left {}, width {}, and height {}".format(self.llv, self.width, self.height)

    @property
    def height(self):
        return self._height
      
    @property
    def width(self):
        return self._width

    @property
    def area(self):
        return self.height * self.width
       
    def transpose(self):
        """Reflection with regard to the line passing through lower left vertex with angle 315 (-45) degrees
        """
        cls = type(self)
        return cls(self.llv, self.height, self.width)

## Solution: Revolving door

In [ ]:
class RevolvingDoor(Door):
    def open(self):
        super().open()
        super().close()

rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

In [23]:
class RevolvingDoor:
    def __init__(self):
        self._door = Door(status=Door.CLOSED)

    def __repr__(self):
        return repr(self._door)
    
    def __getattr__(self, attr):
        return getattr(self._door, attr)
    
    def open(self):
        self._door.open()
        self._door.close()

rdoor = RevolvingDoor()
rdoor.open()
print(rdoor)

Door(closed)


## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com).

The notebook was written using [Python](http://python.org/) 3.7.
Dependencies listed in [environment.yml](../environment.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)